In [1]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import urdf2casadi.urdfparser

import os # For current directory

In [2]:
# urdf2casadi uses cs.MX and cs.SX, which can be hard to read as these are sparse matrices.
# This short function just makes it so that the result will be a numpy matrix
# Use for 
def cs2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()
# NOTE: casadi imports numpy as np, so cs.np is numpy

In [3]:
asd = urdf2casadi.urdfparser.URDFparser()

In [4]:
asd.from_file("./examples/urdf/ur5.urdf")
root_link = "base_link"
end_link = "tool0"
#print asd.robot_desc
fk_dict = asd.get_forward_kinematics(root_link, end_link)
print fk_dict.keys()

./examples/urdf/ur5.urdf
['q', 'upper', 'lower', 'dual_quaternion_fk', 'joint_names', 'T_fk', 'joint_list', 'quaternion_fk']


In [5]:
# CasADi MX symbol giving the joint symbols:
q = fk_dict["q"]
# Upper limits of the joint values
q_upper = fk_dict["upper"]
# Lower limits of the joint values
q_lower = fk_dict["lower"]
# Joint names
joint_names = fk_dict["joint_names"]
print "Number of joints:", q.size()[0]
print "Upper limits:", q_upper
print "Lower limits:", q_lower
print "Joint names:", joint_names

Number of joints: 6
Upper limits: [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718]
Lower limits: [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718]
Joint names: ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']


In [6]:
T0 = fk_dict["T_fk"]([0., 0., 0., 0., 0., 0.])
p0 = T0[:3, 3]
R0 = T0[:3, :3]
print "Transformation matrix:\n",cs2np(T0)
print "Position:\n", "x:",p0[0]," y:", p0[1], " z:", p0[2]
print "Distance from origin:\n", cs.np.linalg.norm(p0), "m"

Transformation matrix:
[[-1.00000000e+00 -9.79317772e-12  4.79531649e-23  8.17250000e-01]
 [ 0.00000000e+00  4.89658886e-12  1.00000000e+00  1.91450000e-01]
 [-9.79317772e-12  1.00000000e+00 -4.89658886e-12 -5.49100000e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Position:
x: 0.81725  y: 0.19145  z: -0.005491
Distance from origin:
0.8393931236807168 m


In [7]:
joint_list, nvar, names, upper, lower = asd.get_joint_info(root_link, end_link)
print(joint_list)
#print(nvar)
#print(names)
#print(upper)
#print(lower)


[<urdf_parser_py.urdf.Joint object at 0x7f8b98b41250>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b41650>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b419d0>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b41d50>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b50110>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b50490>, <urdf_parser_py.urdf.Joint object at 0x7f8b98b56250>]


In [8]:
transformations = asd.get_transforms(root_link, end_link)
print(transformations)
print(len(transformations))
print(len(joint_list))

la
[SX(@1=cos(q_1), @2=sin(q_1), @3=0, @4=0.089159, @5=(-@2), @6=-0.089159, @7=1, 
[[@1, @5, @3, @3, @3, @3], 
 [@2, @1, @3, @3, @3, @3], 
 [@3, @3, @7, @3, @3, @3], 
 [(-(@4*@2)), (@6+(@6*@1)), @3, @1, @5, @3], 
 [(@4+(@4*@1)), (@6*@2), @3, @2, @1, @3], 
 [@3, @3, @3, @3, @3, @7]])]
1
7
